In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import glob
import torch
from tqdm import tqdm



TASK_LIST = [
    # --- 2 Low-Light Tasks (Requires LOE, Entropy, NIQE) ---
    {
        "name": "LowLight_CLAHE", 
        "folder": "../results/CLAHE", 
        "ref_folder": "../dataset", 
        "type": "low_light"
    },
    {
        "name": "LowLight_ZeroDCE",
        "folder": "../results/ZeroDCE",
        "ref_folder": "../dataset",
        "type": "low_light"
    },

    # --- 4 Deblurring/Shake Tasks (Requires Sharpness, Gradient, NIQE) ---
    {
        "name": "CLAHE_Wiener",
        "folder": "../results/CLAHE_Wiener",
        "ref_folder": None, 
        "type": "shake"
    },
    {
        "name": "CLAHE_NAFNet",
        "folder": "../results/CLAHE_NAFNet",
        "ref_folder": None,
        "type": "shake"
    },
    {
        "name": "ZeroDCE_Wiener",
        "folder": "../results/ZeroDCE_Wiener",
        "ref_folder": None,
        "type": "shake"
    },
    {
        "name": "ZeroDCE_NAFNet",
        "folder": "../results/ZeroDCE_NAFNet",
        "ref_folder": None,
        "type": "shake"
    },
]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
try:
    import pyiqa
    print(f"Loading neural metrics (Device: {device})...")
    niqe_net = pyiqa.create_metric('niqe', device=device)
    brisque_net = pyiqa.create_metric('brisque', device=device)
    has_pyiqa = True
except ImportError:
    print("pyiqa library not found. Skipping NIQE/BRISQUE calculation.")
    has_pyiqa = False

def get_entropy(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape)==3 else img
    hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
    hist = hist / hist.sum()
    hist = hist[hist > 0]
    return -np.sum(hist * np.log2(hist))

def get_sharpness(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape)==3 else img
    return cv2.Laplacian(gray, cv2.CV_64F).var()

def get_gradient(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape)==3 else img
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    return np.mean(np.sqrt(sobelx**2 + sobely**2))

def get_loe(img_ref, img_enh, resize=100):
    if img_ref is None or img_enh is None: return np.nan
    L_in = np.max(img_ref, axis=2).astype(np.float32)
    L_out = np.max(img_enh, axis=2).astype(np.float32)
    L_in = cv2.resize(L_in, (resize, resize)).flatten()
    L_out = cv2.resize(L_out, (resize, resize)).flatten()
    order_in = L_in[:, None] >= L_in[None, :]
    order_out = L_out[:, None] >= L_out[None, :]
    diff = np.logical_xor(order_in, order_out)
    return np.sum(diff) / (resize * resize)


all_results = []

print("\nStarting batch processing...")

for task in TASK_LIST:
    task_name = task['name']
    folder = task['folder']
    ref_folder = task['ref_folder']
    task_type = task['type']
    
    # Storage for current task ONLY (for immediate printing)
    current_task_results = []

    if not os.path.exists(folder):
        print(f"Folder not found: {folder} (Skipping)")
        continue

    images = glob.glob(os.path.join(folder, "*"))
    images = [f for f in images if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
    
    print(f"\nProcessing: {task_name} | Type: {task_type} | Images: {len(images)}")
    
    for img_path in tqdm(images, desc="Progress"):
        filename = os.path.basename(img_path)
        img = cv2.imread(img_path)
        if img is None: continue

        row = {
            'Task': task_name,
            'Filename': filename,
            'Type': task_type
        }

        # Neural Metrics
        if has_pyiqa:
            try:
                row['NIQE'] = niqe_net(img_path).item()
                row['BRISQUE'] = brisque_net(img_path).item()
            except:
                row['NIQE'] = np.nan
                row['BRISQUE'] = np.nan

        # Specific Metrics
        if task_type == 'low_light':
            row['Entropy'] = get_entropy(img)
            loe_val = np.nan
            if ref_folder:
                ref_path = os.path.join(ref_folder, filename)
                if not os.path.exists(ref_path):
                    clean_name = filename.replace('output_', '').replace('clahe_', '')
                    ref_path = os.path.join(ref_folder, clean_name)
                
                if os.path.exists(ref_path):
                    ref_img = cv2.imread(ref_path)
                    loe_val = get_loe(ref_img, img)
            
            row['LOE'] = loe_val
            row['Sharpness'] = np.nan
            row['Gradient'] = np.nan

        elif task_type == 'shake':
            row['Sharpness'] = get_sharpness(img)
            row['Gradient'] = get_gradient(img)
            row['Entropy'] = np.nan
            row['LOE'] = np.nan

        # Save to both lists
        all_results.append(row)
        current_task_results.append(row)

    # --- PRINT IMMEDIATE RESULTS FOR THIS TASK ---
    if current_task_results:
        df_task = pd.DataFrame(current_task_results)
        print(f"\nFinished Task: {task_name}")
        print("Average Scores:")
        # Print mean of numeric columns only
        print(df_task.mean(numeric_only=True).to_string())
        print("-" * 50)


if all_results:
    df = pd.DataFrame(all_results)
    
    cols = ['Task', 'Filename', 'Type', 'NIQE', 'BRISQUE', 'Entropy', 'LOE', 'Sharpness', 'Gradient']
    cols = [c for c in cols if c in df.columns]
    df = df[cols]

    csv_name = 'all_results.csv'
    df.to_csv(csv_name, index=False)
    
    print(f"\nAll tasks completed. Detailed results saved to: {csv_name}")
else:
    print("No results generated.")

Loading neural metrics (Device: cuda)...

Starting batch processing...

📂 Processing: LowLight_CLAHE | Type: low_light | Images: 169


Progress: 100%|██████████| 169/169 [01:21<00:00,  2.07it/s]



✅ Finished Task: LowLight_CLAHE
Average Scores:
NIQE                NaN
BRISQUE             NaN
Entropy        2.593962
LOE          260.161083
Sharpness           NaN
Gradient            NaN
--------------------------------------------------

📂 Processing: LowLight_ZeroDCE | Type: low_light | Images: 169


Progress: 100%|██████████| 169/169 [01:22<00:00,  2.05it/s]



✅ Finished Task: LowLight_ZeroDCE
Average Scores:
NIQE                NaN
BRISQUE             NaN
Entropy        2.924774
LOE          303.058136
Sharpness           NaN
Gradient            NaN
--------------------------------------------------

📂 Processing: CLAHE_Wiener | Type: shake | Images: 169


...


✅ Finished Task: CLAHE_NAFNet
Average Scores:
NIQE               NaN
BRISQUE            NaN
Sharpness    13.335887
Gradient      4.371875
Entropy            NaN
LOE                NaN
--------------------------------------------------

📂 Processing: ZeroDCE_Wiener | Type: shake | Images: 169


Progress: 100%|██████████| 169/169 [00:34<00:00,  4.86it/s]



✅ Finished Task: ZeroDCE_Wiener
Average Scores:
NIQE                 NaN
BRISQUE              NaN
Sharpness    1370.770817
Gradient       25.842045
Entropy              NaN
LOE                  NaN
--------------------------------------------------

📂 Processing: ZeroDCE_NAFNet | Type: shake | Images: 169


Progress: 100%|██████████| 169/169 [00:32<00:00,  5.27it/s]


✅ Finished Task: ZeroDCE_NAFNet
Average Scores:
NIQE                NaN
BRISQUE             NaN
Sharpness    803.768436
Gradient      10.720075
Entropy             NaN
LOE                 NaN
--------------------------------------------------

All tasks completed. Detailed results saved to: all_results.csv


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import glob
import torch
from tqdm import tqdm
from PIL import Image  

TASK_LIST = [
    # task1
    {
        "name": "LowLight_CLAHE", 
        "folder": "../results/CLAHE", 
        "ref_folder": "../dataset", 
    },
    {
        "name": "LowLight_ZeroDCE",
        "folder": "../results/ZeroDCE",
        "ref_folder": "../dataset",
    },

    # task2
    {
        "name": "CLAHE_Wiener",
        "folder": "../results/CLAHE_Wiener",
        "ref_folder": None, 
    },
    {
        "name": "CLAHE_NAFNet",
        "folder": "../results/CLAHE_NAFNet",
        "ref_folder": None,
    },
    {
        "name": "ZeroDCE_Wiener",
        "folder": "../results/ZeroDCE_Wiener",
        "ref_folder": None,
    },
    {
        "name": "ZeroDCE_NAFNet",
        "folder": "../results/ZeroDCE_NAFNet",
        "ref_folder": None,
    },
]

# preprocess


if torch.cuda.is_available() and torch.cuda.device_count() > 2:
    device = torch.device("cuda:2")
    print("Using GPU: cuda:2")
else:

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"'cuda:2' not available. Fallback to: {device}")


try:
    import pyiqa
    print("Loading NIQE model...")
    niqe_net = pyiqa.create_metric('niqe', device=device)
    
    print("Loading BRISQUE model...")
    brisque_net = pyiqa.create_metric('brisque', device=device)
    
    print("Loading MANIQA model...")
    maniqa_net = pyiqa.create_metric('maniqa', device=device)
    
    has_pyiqa = True
    print("All Neural Metrics Loaded Successfully.")
except ImportError:
    print("pyiqa not installed. NIQE/BRISQUE/MANIQA will be NaN.")
    has_pyiqa = False
except Exception as e:
    print(f"Error loading pyiqa models: {e}")
    has_pyiqa = False


def compute_tenengrad(img):

    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img
    gx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    gy = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    return np.mean(gx**2 + gy**2)

def compute_entropy(img):

    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img
    hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
    hist = hist / hist.sum()
    hist = hist[hist > 0]
    return -np.sum(hist * np.log2(hist))

def compute_loe(input_img, enhanced_img, resize_dim=100):

    L_in = np.max(input_img, axis=2).astype(np.float32)
    L_out = np.max(enhanced_img, axis=2).astype(np.float32)

    L_in = cv2.resize(L_in, (resize_dim, resize_dim))
    L_out = cv2.resize(L_out, (resize_dim, resize_dim))

    vec_in = L_in.flatten()
    vec_out = L_out.flatten()

    order_in = vec_in[:, None] >= vec_in[None, :]
    order_out = vec_out[:, None] >= vec_out[None, :]

    difference_matrix = np.logical_xor(order_in, order_out)
    return np.mean(np.sum(difference_matrix, axis=1))


# Main Loop

all_results = []
print("\nStarting evaluation...")

for task in TASK_LIST:
    task_name = task['name']
    folder = task['folder']
    ref_folder = task['ref_folder']
    
    current_task_rows = []

    if not os.path.exists(folder):
        print(f"Skipping {task_name}: Folder not found ({folder})")
        continue

    images = glob.glob(os.path.join(folder, "*"))
    images = [f for f in images if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
    
    print(f"\nProcessing {task_name} ({len(images)} images)...")

    for img_path in tqdm(images):
        filename = os.path.basename(img_path)
        
        img_cv = cv2.imread(img_path)
        if img_cv is None: continue

        row = {
            'Task': task_name,
            'Filename': filename
        }


        row['Tenengrad'] = compute_tenengrad(img_cv)
        row['Entropy'] = compute_entropy(img_cv)


        img_pil = Image.open(img_path).convert('RGB')
        row['NIQE'] = niqe_net(img_pil).item()
        row['BRISQUE'] = brisque_net(img_pil).item()
        row['MANIQA'] = maniqa_net(img_pil).item()

        

        row['LOE'] = np.nan 
        
        if ref_folder:
            ref_path = os.path.join(ref_folder, filename)
            if not os.path.exists(ref_path):
                clean_name = filename.replace('output_', '').replace('clahe_', '')
                ref_path = os.path.join(ref_folder, clean_name)
            
            if os.path.exists(ref_path):
                img_ref = cv2.imread(ref_path)
                if img_ref is not None:
                    row['LOE'] = compute_loe(img_ref, img_cv)

        all_results.append(row)
        current_task_rows.append(row)

    if current_task_rows:
        df_curr = pd.DataFrame(current_task_rows)
        print(f"Finished Task: {task_name}")
        print("Average Scores:")
        print(df_curr.mean(numeric_only=True).to_string())

# save results

if all_results:
    df_final = pd.DataFrame(all_results)
    
    cols = ['Task', 'Filename', 'NIQE', 'BRISQUE', 'MANIQA', 'Entropy', 'Tenengrad', 'LOE']
    cols = [c for c in cols if c in df_final.columns]
    df_final = df_final[cols]

    csv_name = 'final_evaluation_results.csv'
    df_final.to_csv(csv_name, index=False)
    
    print(f"\nAll Done! Results saved to: {csv_name}")
else:
    print("No images processed.")

🚀 Using GPU: cuda:2
Loading NIQE model...
Loading BRISQUE model...
Loading MANIQA model...
Loading pretrained model MANIQA from /home/pearson/.cache/torch/hub/pyiqa/ckpt_koniq10k.pt
✅ All Neural Metrics Loaded Successfully.

Starting evaluation...

📂 Processing LowLight_CLAHE (169 images)...


100%|██████████| 169/169 [03:33<00:00,  1.26s/it]


✅ Finished Task: LowLight_CLAHE
Average Scores:
Tenengrad    454.331793
Entropy        2.593962
NIQE           8.542116
BRISQUE       92.501018
MANIQA         0.255225
LOE          260.161083

📂 Processing LowLight_ZeroDCE (169 images)...


100%|██████████| 169/169 [03:33<00:00,  1.26s/it]


✅ Finished Task: LowLight_ZeroDCE
Average Scores:
Tenengrad    1268.406952
Entropy         2.924774
NIQE            6.889668
BRISQUE        69.022779
MANIQA          0.238043
LOE           303.058136

📂 Processing CLAHE_Wiener (169 images)...


...

100%|██████████| 169/169 [02:35<00:00,  1.09it/s]


✅ Finished Task: ZeroDCE_Wiener
Average Scores:
Tenengrad    5120.891045
Entropy         3.782977
NIQE            9.723525
BRISQUE        95.343323
MANIQA          0.290217
LOE                  NaN

📂 Processing ZeroDCE_NAFNet (169 images)...


100%|██████████| 169/169 [02:30<00:00,  1.12it/s]

✅ Finished Task: ZeroDCE_NAFNet
Average Scores:
Tenengrad    2940.527331
Entropy         3.023814
NIQE            5.727641
BRISQUE        91.511644
MANIQA          0.288861
LOE                  NaN

🎉 All Done! Results saved to: final_evaluation_results.csv


: 